### Work in progress
Test och se om vi kan bättre utnyttja fält i Litteraturbankens json
* sbl_link --> Wikidata Property 3217 = Svenskt Biografiskt Lexikon
* skbl_link --> Wikidata Property 4963 = Svenskt kvinnobiografiskt lexikon
* sol_link ?!?!?!
* wikidata_id = Wikidata Qnummer
* wikipedia = WIkipedia URL


Denna [Notebook](https://github.com/salgo60/spa2Commons/blob/main/Notebook/Literaturbanken%20-%20samma%20som.ipynb) 
* [issue 3](https://github.com/salgo60/spa2Commons/issues/3) om vi kan bättre förstå hur Litteraturbanken arbetar

**TODO**
 -  [ ] kontrollera data
 -  [ ] förstå vad SOL är
 - [ ] bättre koll diff SBL SKBL skriva ut till fil
 - [ ] kolla samma som LIBRISXL



Status


| Date | Litteraturbanken show | Littbank - WD | Littbank - SBL | Littbank - SKBL|WD - Littbank | WD - Littbank - SBL  |WD - Littbank - SKBL  |
| ------------- |:-------------:|:-------------:|:-------------:|:-------------:|:-------------:|:-------------:| -----:|
| 20211001 | 3130 | 2132 | 765 | 152 | 2346 | 810 | 161



In [30]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2021-10-03 13:07:46.875200


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 
pd.set_option("display.max.columns", None) 
 
url = "https://litteraturbanken.se/api/get_authors"
r = http.request('GET', url)
data = json.loads(r.data)
df = pd.json_normalize(data["data"])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Data columns (total 50 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   authorid                       4024 non-null   object 
 1   authorid_norm                  4024 non-null   object 
 2   db_checksum                    4024 non-null   object 
 3   db_timestamp                   4024 non-null   int64  
 4   doc_type                       4024 non-null   object 
 5   full_name                      4024 non-null   object 
 6   gender                         4024 non-null   object 
 7   imported                       3365 non-null   object 
 8   intro                          728 non-null    object 
 9   librisid                       2166 non-null   object 
 10  name_for_index                 4024 non-null   object 
 11  pictureinfo                    169 non-null    object 
 12  searchable                     4024 non-null   b

In [4]:
df["show"].value_counts()

True     3130
False     894
Name: show, dtype: int64

In [5]:
#just objects with show
dfShow = df[df["show"]].copy() 

In [6]:
dfexternal = dfShow[{"authorid","wikidata.sbl_link","wikidata.skbl_link","wikidata.wikidata_id","wikidata.sol_link"
,"wikidata.wikidata_id","wikidata.wikipedia"}]

In [7]:
dfexternal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3130 entries, 1 to 4023
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   authorid              3130 non-null   object
 1   wikidata.sol_link     118 non-null    object
 2   wikidata.wikipedia    1752 non-null   object
 3   wikidata.skbl_link    152 non-null    object
 4   wikidata.sbl_link     765 non-null    object
 5   wikidata.wikidata_id  2134 non-null   object
dtypes: object(6)
memory usage: 171.2+ KB


In [29]:
dfexternal["wikidata.sol_link"].value_counts()

Karin_Jensen                   1
Albert_Eriksson                1
Walborg_Hedberg                1
Jonas_Magnus_Stjernstolpe      1
Nicolaus_Balk                  1
                              ..
Östen_Sjöstrand                1
Vera_von_Kræmer                1
Ulrica_Carolina_Widström       1
Ericus_Benedicti_Schroderus    1
Christopher_Manderström        1
Name: wikidata.sol_link, Length: 118, dtype: int64

In [8]:
dfexternal

,authorid,wikidata.sol_link,wikidata.wikipedia,wikidata.skbl_link,wikidata.sbl_link,wikidata.wikidata_id
1,AasenE,None,None,None,None,Q11967131
2,AbeniusM,None,https://sv.wikipedia.org/wiki/Margit_Abenius,MargitAbenius,None,Q4933592
3,AbrahamssonA,NaN,NaN,NaN,NaN,NaN
5,AbrahamssonS,None,https://sv.wikipedia.org/wiki/Selma_Abrahamsson,None,None,Q4934135
6,AchariusE,None,https://sv.wikipedia.org/wiki/Erik_Acharius,None,5503,Q365923
...,...,...,...,...,...,...
4015,ÖnnerforsA,None,https://sv.wikipedia.org/wiki/Alf_%C3%96nnerfors,None,None,Q1237808
4017,ÖstergrenCL,None,https://sv.wikipedia.org/wiki/Carl_%C3%96stergren,None,None,Q6257795
4018,ÖstergrenC,NaN,NaN,NaN,NaN,NaN
4021,ÖstinO,None,None,None,None,Q100752816


In [9]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ass.ms) {}'.format(datetime.now() - start_time))

Ended:  2021-10-03 13:00:52.728760
Time elapsed (hh:mm:ass.ms) 0:00:02.271924


### Get Wikidata

In [10]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys,json
import pandas as pd 

from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# https://w.wiki/4AAV
query = """SELECT (REPLACE(STR(?item), ".*Q", "Q") AS ?WikidataID)  ?authorid ?SBL ?SKBL WHERE {
?item wdt:P31 wd:Q5.
?item wdt:P5101 ?authorid
OPTIONAL {?item wdt:P3217 ?SBL}
OPTIONAL {?item wdt:P4963 ?SKBL}
} order by ?authorid"""
 
    
def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']
    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDLittbanktot = get_sparql_dataframe(endpoint_url, query)


In [11]:
WDLittbanktot['SBL'] = pd.to_numeric(WDLittbanktot['SBL'], errors="coerce")
WDLittbanktot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2349 entries, 0 to 2348
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   WikidataID  2349 non-null   object 
 1   authorid    2349 non-null   object 
 2   SBL         811 non-null    float64
 3   SKBL        161 non-null    object 
dtypes: float64(1), object(3)
memory usage: 73.5+ KB


In [12]:
dfexternal['wikidata.sbl_link'] = pd.to_numeric(dfexternal['wikidata.sbl_link'], errors="coerce")
dfexternal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3130 entries, 1 to 4023
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   authorid              3130 non-null   object 
 1   wikidata.sol_link     118 non-null    object 
 2   wikidata.wikipedia    1752 non-null   object 
 3   wikidata.skbl_link    152 non-null    object 
 4   wikidata.sbl_link     765 non-null    float64
 5   wikidata.wikidata_id  2134 non-null   object 
dtypes: float64(1), object(5)
memory usage: 171.2+ KB


<ipython-input-12-e1d483007a37>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfexternal['wikidata.sbl_link'] = pd.to_numeric(dfexternal['wikidata.sbl_link'], errors="coerce")


### Compare Littbank Wikidata field with Wikidata 


In [13]:
WDLittbank_WD_merge =  pd.merge(WDLittbanktot, dfexternal, on='authorid',indicator=True)
WDLittbank_WD_merge.rename(columns={"_merge": "WD_Littbank_merge"},inplace = True)

WDLittbank_WD_merge["WD_Littbank_merge"].value_counts()

both          2348
left_only        0
right_only       0
Name: WD_Littbank_merge, dtype: int64

In [14]:
WDLittbank_WD_merge["WD_Littbank_merge"].value_counts()

both          2348
left_only        0
right_only       0
Name: WD_Littbank_merge, dtype: int64

In [15]:
# all with SBL seems to have WIkidata
WDLittbanktotSBL =  WDLittbank_WD_merge[~WDLittbank_WD_merge['wikidata.sbl_link'].isnull()]
WDLittbanktotSBL_noWD = WDLittbanktotSBL[WDLittbanktotSBL['wikidata.wikidata_id'].isnull()] 
WDLittbanktotSBL_noWD

,WikidataID,authorid,SBL,SKBL,wikidata.sol_link,wikidata.wikipedia,wikidata.skbl_link,wikidata.sbl_link,wikidata.wikidata_id,WD_Littbank_merge


### SBL

In [16]:
WDLittbanktot_SBL = WDLittbanktot[~WDLittbanktot["SBL"].isna()]
WDLittbanktot_SBL.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 811 entries, 5 to 2280
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   WikidataID  811 non-null    object 
 1   authorid    811 non-null    object 
 2   SBL         811 non-null    float64
 3   SKBL        79 non-null     object 
dtypes: float64(1), object(3)
memory usage: 31.7+ KB


In [17]:
dfexternal_SBL = dfexternal[~dfexternal["wikidata.sbl_link"].isna()]

In [18]:
dfexternal_SBL.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 765 entries, 6 to 3897
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   authorid              765 non-null    object 
 1   wikidata.sol_link     66 non-null     object 
 2   wikidata.wikipedia    763 non-null    object 
 3   wikidata.skbl_link    77 non-null     object 
 4   wikidata.sbl_link     765 non-null    float64
 5   wikidata.wikidata_id  765 non-null    object 
dtypes: float64(1), object(5)
memory usage: 41.8+ KB


In [19]:
WDLittbank_WD_SBL_merge =  pd.merge(WDLittbanktot_SBL, dfexternal_SBL, left_on='SBL', right_on='wikidata.sbl_link',indicator=True)
WDLittbank_WD_SBL_merge.rename(columns={"_merge": "WD_Littbank_SBL_merge"},inplace = True)

WDLittbank_WD_SBL_merge["WD_Littbank_SBL_merge"].value_counts()

both          766
left_only       0
right_only      0
Name: WD_Littbank_SBL_merge, dtype: int64

## SKBL

In [20]:
WDLittbanktot_SKBL = WDLittbanktot[~WDLittbanktot["SKBL"].isna()]
WDLittbanktot_SKBL.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161 entries, 2 to 2338
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   WikidataID  161 non-null    object 
 1   authorid    161 non-null    object 
 2   SBL         79 non-null     float64
 3   SKBL        161 non-null    object 
dtypes: float64(1), object(3)
memory usage: 6.3+ KB


In [21]:
dfexternal_SKBL = dfexternal[~dfexternal["wikidata.skbl_link"].isna()]
dfexternal_SKBL.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152 entries, 2 to 4006
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   authorid              152 non-null    object 
 1   wikidata.sol_link     23 non-null     object 
 2   wikidata.wikipedia    152 non-null    object 
 3   wikidata.skbl_link    152 non-null    object 
 4   wikidata.sbl_link     77 non-null     float64
 5   wikidata.wikidata_id  152 non-null    object 
dtypes: float64(1), object(5)
memory usage: 8.3+ KB


In [22]:
WDLittbank_WD_SKBL_merge =  pd.merge(WDLittbanktot_SKBL, dfexternal_SBL,how='outer', left_on='SKBL', right_on='wikidata.skbl_link',indicator=True)
WDLittbank_WD_SKBL_merge.rename(columns={"_merge": "WD_Littbank_SKBL_merge"},inplace = True)

WDLittbank_WD_SKBL_merge["WD_Littbank_SKBL_merge"].value_counts()

right_only    688
left_only      84
both           77
Name: WD_Littbank_SKBL_merge, dtype: int64

In [23]:
WDLittbank_WD_SKBL_merge

,WikidataID,authorid_x,SBL,SKBL,authorid_y,wikidata.sol_link,wikidata.wikipedia,wikidata.skbl_link,wikidata.sbl_link,wikidata.wikidata_id,WD_Littbank_SKBL_merge
0,Q4933592,AbeniusM,NaN,MargitAbenius,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,Q4933819,AdelborgO,5519.0,OttiliaAdelborg,AdelborgO,None,https://sv.wikipedia.org/wiki/Ottilia_Adelborg,OttiliaAdelborg,5519.0,Q4933819,both
2,Q4346827,AdlersparreS,5564.0,SophieAdlersparre,AdlersparreS,None,https://sv.wikipedia.org/wiki/Sophie_Adlersparre,SophieAdlersparre,5564.0,Q4346827,both
3,Q4933929,AdolfssonE,NaN,EvaAdolfsson,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,Q469339,AgrellA,5599.0,AlfhildAgrell,AgrellA,None,https://sv.wikipedia.org/wiki/Alfhild_Agrell,AlfhildAgrell,5599.0,Q469339,both
...,...,...,...,...,...,...,...,...,...,...,...
844,NaN,NaN,NaN,NaN,SvedeliusWE,None,https://sv.wikipedia.org/wiki/Wilhelm_Erik_Sve...,None,34837.0,Q6197780,right_only
845,NaN,NaN,NaN,NaN,SwedbergJ,None,https://sv.wikipedia.org/wiki/Jesper_Swedberg,None,34820.0,Q748553,right_only
846,NaN,NaN,NaN,NaN,SwedenborgE,None,https://sv.wikipedia.org/wiki/Emanuel_Swedenborg,None,34840.0,Q185832,right_only
847,NaN,NaN,NaN,NaN,SwederusG,None,https://sv.wikipedia.org/wiki/Georg_Swederus,None,34844.0,Q22110411,right_only


In [24]:
print("|",start_time.strftime("%Y%m%d"),"|",dfShow["authorid"].nunique(),
     "|",dfShow["wikidata.wikidata_id"].nunique(),
     "|",dfShow["wikidata.sbl_link"].nunique(),
     "|",dfShow["wikidata.skbl_link"].nunique(),
     "|",WDLittbanktot["authorid"].nunique(),
     "|",WDLittbanktot["SBL"].nunique(),
     "|",WDLittbanktot["SKBL"].nunique()
     )

| 20211003 | 3130 | 2132 | 765 | 152 | 2347 | 810 | 161


In [25]:
#dfShow["wikidata.wikidata_id"].nunique()

In [26]:
#WDLittbanktot.info()

In [27]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2021-10-03 13:00:54.854411
Time elapsed (hh:mm:ss.ms) 0:00:04.397581
